### Init

In [1]:
import kuti
from kuti import model_helper as mh
from kuti import applications as apps
from kuti import tensor_ops as ops
from kuti import generic as gen
import pandas as pd, numpy as np, os

Loaded Kuti


# GET2 Files training

## Train the network on GET2 files

In [2]:
root_path = '/media/workstation/0832621B32620DCE/Ian/'
dataset = root_path + 'mtaiq/PARA_MTAIQ_GET2_official_dataset.csv'
ids = pd.read_csv(dataset)

print('ID for PARA Dataset')
ids

# input_shape = (None, None, 3)
# features_root = root_path + 'features_get2/'
# images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA'

ID for PARA Dataset


,sessionId,imageName,aestheticScore,qualityScore,sessionId_imageName,set
0,session1,iaa_pub1_.jpg,3.809524,3.923810,session1_iaa_pub1_.jpg,training
1,session1,iaa_pub2_.jpg,3.142857,3.385714,session1_iaa_pub2_.jpg,training
2,session1,iaa_pub3_.jpg,2.928571,3.128571,session1_iaa_pub3_.jpg,training
3,session1,iaa_pub4_.jpg,3.428571,3.752381,session1_iaa_pub4_.jpg,training
4,session1,iaa_pub5_.jpg,3.214286,3.538095,session1_iaa_pub5_.jpg,training
...,...,...,...,...,...,...
31215,session340,iaa_pub23744_.jpg,3.500000,3.608333,session340_iaa_pub23744_.jpg,validation
31216,session137,iaa_pub9552_.jpg,3.425000,3.720000,session137_iaa_pub9552_.jpg,validation
31217,session256,iaa_pub17866_.jpg,3.023810,3.233333,session256_iaa_pub17866_.jpg,validation
31218,session134,iaa_pub9349_.jpg,4.022727,4.154545,session134_iaa_pub9349_.jpg,validation


### Train on MLSP wide features (aestheticScore only)

In [6]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_get2_baseline_aestheticScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_aesthetic_out = 'MSE')
loss_weights = dict(head_aesthetic_out = 1.0)
metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['aestheticScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
                                fc_sizes      = fc_sizes,
                                dropout_rates = dropout_rates,
                                batch_norm    = bn)

# pred_quality = apps.fc_layers(x, name       = 'head_quality',
#                               fc_sizes      = fc_sizes,
#                               dropout_rates = dropout_rates,
#                               batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_aesthetic) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_get2_aestheticScore_baseline',
                     models_root    = root_path + 'models_get2_aestheticScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


In [7]:
for lr in [1e-4,1e-5,1e-6]:
    helper.load_model()
    helper.train(lr=lr, epochs=20)

Model NOT loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5 does not exist
Training model: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]
Epoch 1/20


2023-02-01 15:57:08.417639: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-02-01 15:57:14.124277: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


170/170 [==============================] - 720s 4s/step - loss: 11.8891 - MAE: 3.0640 - plcc_tf: 0.2202 - val_loss: 8.0581 - val_MAE: 2.8010 - val_plcc_tf: 0.5643
Epoch 2/20
170/170 [==============================] - 645s 4s/step - loss: 9.6040 - MAE: 2.7635 - plcc_tf: 0.2574 - val_loss: 6.1734 - val_MAE: 2.4395 - val_plcc_tf: 0.6311
Epoch 3/20
170/170 [==============================] - 508s 3s/step - loss: 7.9656 - MAE: 2.4932 - plcc_tf: 0.2734 - val_loss: 4.9948 - val_MAE: 2.1947 - val_plcc_tf: 0.7057
Epoch 4/20
170/170 [==============================] - 371s 2s/step - loss: 6.5641 - MAE: 2.2317 - plcc_tf: 0.2755 - val_loss: 4.0959 - val_MAE: 1.9710 - val_plcc_tf: 0.6895
Epoch 5/20
170/170 [==============================] - 412s 2s/step - loss: 5.2981 - MAE: 1.9564 - plcc_tf: 0.2805 - val_loss: 2.8016 - val_MAE: 1.6147 - val_plcc_tf: 0.6729
Epoch 6/20
170/170 [==============================] - 360s 2s/step - loss: 4.2814 - MAE: 1.7155 - plcc_tf: 0.2629 - val_loss: 1.7735 - val_MAE: 1

#### Test without augmentation

In [5]:
print('After training')

helper.load_model(root_path + 'models/irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE,MSE] mon:[val_head_aesthetic_out_plcc_tf] o:2[1]')
test_gen = helper.make_generator(ids[ids.set=='test'])
helper.validate(test_gen, verbose=2)



After training
Model weights loaded: /media/workstation/0832621B32620DCE/Ian/models/irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE,MSE] mon:[val_head_aesthetic_out_plcc_tf] o:2[1]_best_weights.h5
Validating performance


2023-01-31 13:50:06.698276: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-01-31 13:50:14.044455: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


36/36 - 180s - loss: 0.1228 - head_aesthetic_out_loss: 0.0621 - head_quality_out_loss: 0.0607 - head_aesthetic_out_MAE: 0.1966 - head_aesthetic_out_plcc_tf: 0.9137 - head_quality_out_MAE: 0.1931 - head_quality_out_plcc_tf: 0.9188 - 180s/epoch - 5s/step
head_aesthetic_out_MAE:     0.19663752615451813
head_aesthetic_out_loss:    0.062127988785505295
head_aesthetic_out_plcc_tf: 0.9137099385261536
head_quality_out_MAE:       0.19310589134693146
head_quality_out_loss:      0.060700468719005585
head_quality_out_plcc_tf:   0.9187530875205994
loss:                       0.12282846868038177


{'loss': 0.12282846868038177,
 'head_aesthetic_out_loss': 0.062127988785505295,
 'head_quality_out_loss': 0.060700468719005585,
 'head_aesthetic_out_MAE': 0.19663752615451813,
 'head_aesthetic_out_plcc_tf': 0.9137099385261536,
 'head_quality_out_MAE': 0.19310589134693146,
 'head_quality_out_plcc_tf': 0.9187530875205994}

### Train on MLSP wide features (qualityScore only)

In [3]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_get2_baseline_qualityScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_quality_out = 'MSE')
loss_weights = dict(head_quality_out = 1.0)
metrics      = dict(head_quality_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['qualityScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

# pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
#                                 fc_sizes      = fc_sizes,
#                                 dropout_rates = dropout_rates,
#                                 batch_norm    = bn)

pred_quality = apps.fc_layers(x, name       = 'head_quality',
                              fc_sizes      = fc_sizes,
                              dropout_rates = dropout_rates,
                              batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_quality) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_get2_qualityScore_retrain_baseline',
                     models_root    = root_path + 'models_get2_qualityScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

2023-02-05 20:45:53.274957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 20:45:53.926657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 20:45:53.926864: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 20:45:53.927350: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


In [4]:
for lr in [1e-4,1e-5,1e-6]:
    helper.load_model()
    helper.train(lr=lr, epochs=20)

Model NOT loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5 does not exist
Training model: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]
Epoch 1/20


2023-02-05 20:46:10.753156: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-02-05 20:46:17.520056: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


170/170 [==============================] - 317s 2s/step - loss: 12.8811 - MAE: 3.2265 - plcc_tf: 0.2302 - val_loss: 8.8822 - val_MAE: 2.9303 - val_plcc_tf: 0.4034
Epoch 2/20
170/170 [==============================] - 333s 2s/step - loss: 10.5955 - MAE: 2.9462 - plcc_tf: 0.2629 - val_loss: 6.7568 - val_MAE: 2.5644 - val_plcc_tf: 0.6987
Epoch 3/20
170/170 [==============================] - 354s 2s/step - loss: 8.7738 - MAE: 2.6531 - plcc_tf: 0.2770 - val_loss: 6.0636 - val_MAE: 2.4210 - val_plcc_tf: 0.6973
Epoch 4/20
170/170 [==============================] - 335s 2s/step - loss: 7.2233 - MAE: 2.3696 - plcc_tf: 0.2757 - val_loss: 5.1662 - val_MAE: 2.2360 - val_plcc_tf: 0.7345
Epoch 5/20
170/170 [==============================] - 372s 2s/step - loss: 5.8981 - MAE: 2.0927 - plcc_tf: 0.2827 - val_loss: 4.0653 - val_MAE: 1.9753 - val_plcc_tf: 0.7683
Epoch 6/20
170/170 [==============================] - 451s 3s/step - loss: 4.6163 - MAE: 1.8057 - plcc_tf: 0.2838 - val_loss: 2.5257 - val_MAE: 

#### Test without augmentation

In [5]:
print('After training')

helper.load_model()
test_gen = helper.make_generator(ids[ids.set=='test'])
helper.validate(test_gen, verbose=2)

After training
Model weights loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET2_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5
Validating performance
36/36 - 124s - loss: 0.0687 - MAE: 0.2075 - plcc_tf: 0.9129 - 124s/epoch - 3s/step
MAE:     0.20753511786460876
loss:    0.06866291910409927
plcc_tf: 0.9129168391227722


{'loss': 0.06866291910409927,
 'MAE': 0.20753511786460876,
 'plcc_tf': 0.9129168391227722}

# GET3 Files training


## Train the network on GET3 files 

In [3]:

root_path = '/media/workstation/0832621B32620DCE/Ian/'
dataset = root_path + 'mtaiq/PARA_MTAIQ_GET3_official_dataset.csv'
ids = pd.read_csv(dataset)

print('ID for PARA Dataset')
ids

# input_shape = (None, None, 3)
# features_root = root_path + 'features_get3/'
# images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA'

ID for PARA Dataset


,sessionId,imageName,aestheticScore,qualityScore,sessionId_imageName,set
0,session1,iaa_pub2_.jpg,3.104167,3.341667,session1_iaa_pub2_.jpg,training
1,session1,iaa_pub5_.jpg,3.187500,3.491667,session1_iaa_pub5_.jpg,training
2,session1,iaa_pub6_.jpg,3.562500,3.866667,session1_iaa_pub6_.jpg,training
3,session1,iaa_pub9_.jpg,2.625000,2.904167,session1_iaa_pub9_.jpg,training
4,session1,iaa_pub10_.jpg,2.895833,3.158333,session1_iaa_pub10_.jpg,training
...,...,...,...,...,...,...
31215,session5,iaa_pub296_.jpg,3.840000,3.952000,session5_iaa_pub296_.jpg,validation
31216,session14,iaa_pub922_.jpg,4.152174,4.226087,session14_iaa_pub922_.jpg,validation
31217,session331,iaa_pub23123_.jpg,3.820000,3.848000,session331_iaa_pub23123_.jpg,validation
31218,session225,iaa_pub15749_.jpg,3.041667,3.150000,session225_iaa_pub15749_.jpg,validation


### Train on MLSP wide features (aestheticScore)

In [10]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_get3_baseline_aestheticsScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_aesthetic_out = 'MSE')
loss_weights = dict(head_aesthetic_out = 1.0)
metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

# monitor_metric = 'val_plcc_tf'; 
monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['aestheticScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
                                fc_sizes      = fc_sizes,
                                dropout_rates = dropout_rates,
                                batch_norm    = bn)

# pred_quality = apps.fc_layers(x, name       = 'head_quality',
#                               fc_sizes      = fc_sizes,
#                               dropout_rates = dropout_rates,
#                               batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_aesthetic) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_get3_aestheticScore_baseline',
                     models_root    = root_path + 'models_get3_aestheticScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET3_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


### Train on MLSP wide features (qualityScore)

In [4]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_get3_baseline_qualityScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_quality_out = 'MSE')
loss_weights = dict(head_quality_out = 1.0)
metrics      = dict(head_quality_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['qualityScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

# pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
#                                 fc_sizes      = fc_sizes,
#                                 dropout_rates = dropout_rates,
#                                 batch_norm    = bn)

pred_quality = apps.fc_layers(x, name       = 'head_quality',
                              fc_sizes      = fc_sizes,
                              dropout_rates = dropout_rates,
                              batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_quality) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_get3_qualityScore_baseline',
                     models_root    = root_path + 'models_get3_aestheticScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

2023-02-01 07:17:06.159372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET3_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


2023-02-01 07:17:06.197251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 07:17:06.197406: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 07:17:06.198394: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 07:17:06.198883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

In [5]:
for lr in [1e-4,1e-5,1e-6]:
    helper.load_model()
    helper.train(lr=lr, epochs=20)

Model NOT loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET3_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5 does not exist
Training model: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET3_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]
Epoch 1/20


2023-02-01 07:17:49.217855: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-02-01 07:17:49.609646: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


170/170 [==============================] - 789s 5s/step - loss: 12.6764 - MAE: 3.2077 - plcc_tf: 0.2337 - val_loss: 8.4352 - val_MAE: 2.8468 - val_plcc_tf: 0.3229
Epoch 2/20
170/170 [==============================] - 713s 4s/step - loss: 10.2765 - MAE: 2.9030 - plcc_tf: 0.2687 - val_loss: 7.0993 - val_MAE: 2.6254 - val_plcc_tf: 0.6558
Epoch 3/20
170/170 [==============================] - 459s 3s/step - loss: 8.6412 - MAE: 2.6383 - plcc_tf: 0.2752 - val_loss: 5.5962 - val_MAE: 2.3135 - val_plcc_tf: 0.6507
Epoch 4/20
170/170 [==============================] - 413s 2s/step - loss: 7.1204 - MAE: 2.3579 - plcc_tf: 0.2827 - val_loss: 4.2475 - val_MAE: 2.0194 - val_plcc_tf: 0.7652
Epoch 5/20
170/170 [==============================] - 339s 2s/step - loss: 5.7935 - MAE: 2.0798 - plcc_tf: 0.2877 - val_loss: 3.3531 - val_MAE: 1.7955 - val_plcc_tf: 0.7799
Epoch 6/20
170/170 [==============================] - 342s 2s/step - loss: 4.6548 - MAE: 1.8129 - plcc_tf: 0.2877 - val_loss: 2.1647 - val_MAE: 

### Test without augmentation

In [6]:
print('After training')

helper.load_model()
test_gen = helper.make_generator(ids[ids.set=='test'])
helper.validate(test_gen, verbose=2)

After training
Model weights loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_GET3_official_datase fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5
Validating performance
36/36 - 155s - loss: 0.0578 - MAE: 0.1886 - plcc_tf: 0.9144 - 155s/epoch - 4s/step
MAE:     0.18862153589725494
loss:    0.057820480316877365
plcc_tf: 0.9144100546836853


{'loss': 0.057820480316877365,
 'MAE': 0.18862153589725494,
 'plcc_tf': 0.9144100546836853}

# All User Files training


In [2]:

root_path = '/media/workstation/0832621B32620DCE/Ian/'
dataset = root_path + 'mtaiq/PARA_MTAIQ_All_User_official_dataset.csv'
ids = pd.read_csv(dataset)

print('ID for PARA Dataset')
ids

# input_shape = (None, None, 3)
# features_root = root_path + 'features_get3/'
# images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA'

ID for PARA Dataset


,sessionId,imageName,aestheticScore,qualityScore,sessionId_imageName,set
0,session328,iaa_pub22893_.jpg,1.980000,2.084000,session328_iaa_pub22893_.jpg,training
1,session232,iaa_pub16180_.jpg,3.720000,3.840000,session232_iaa_pub16180_.jpg,training
2,session125,iaa_pub8721_.jpg,2.900000,3.052000,session125_iaa_pub8721_.jpg,training
3,session139,iaa_pub9725_.jpg,3.120000,3.356000,session139_iaa_pub9725_.jpg,training
4,session79,iaa_pub5474_.jpg,3.840000,3.932000,session79_iaa_pub5474_.jpg,training
...,...,...,...,...,...,...
31215,session7,iaa_pub484_.jpg,3.540000,3.664000,session7_iaa_pub484_.jpg,validation
31216,session68,iaa_pub4700_.jpg,2.640000,2.984000,session68_iaa_pub4700_.jpg,validation
31217,session20,iaa_pub1377_.jpg,3.340000,3.428000,session20_iaa_pub1377_.jpg,validation
31218,session64,iaa_pub4477_.jpg,3.464286,3.607143,session64_iaa_pub4477_.jpg,validation


## Train on MLSP Wide Features (aestheticScore only)

In [3]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_all_user_baseline_aestheticsScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_aesthetic_out = 'MSE')
loss_weights = dict(head_aesthetic_out = 1.0)
metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

# monitor_metric = 'val_plcc_tf'; 
monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['aestheticScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
                                fc_sizes      = fc_sizes,
                                dropout_rates = dropout_rates,
                                batch_norm    = bn)

# pred_quality = apps.fc_layers(x, name       = 'head_quality',
#                               fc_sizes      = fc_sizes,
#                               dropout_rates = dropout_rates,
#                               batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_aesthetic) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_all_user_aestheticScore_baseline',
                     models_root    = root_path + 'models_all_user_aestheticScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


2023-02-05 02:51:25.307787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 02:51:25.502751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 02:51:25.502924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 02:51:25.503405: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

## Train on MLSP Wide Features (qualityScore only)

In [3]:
# features_file = root_path + 'features/irnv2_mlsp_wide_orig/i1[orig]_lfinal_o1[5,5,16928]_r1.h5'
features_file = root_path + 'features_all_user_baseline_qualityScore/irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5'

fc1_size = 2048
image_size = '[orig]'
input_size = (5,5,16928)
model_name = features_file.split('/')[-2]

# loss         = dict(head_aesthetic_out = 'MSE', head_quality_out = 'MSE')
# loss_weights = dict(head_aesthetic_out = 1.0,   head_quality_out = 1.0)
# metrics      = dict(head_aesthetic_out = ['MAE', ops.plcc_tf], head_quality_out = ['MAE', ops.plcc_tf])

loss         = dict(head_quality_out = 'MSE')
loss_weights = dict(head_quality_out = 1.0)
metrics      = dict(head_quality_out = ['MAE', ops.plcc_tf])

bn = 2
fc_sizes = [fc1_size, fc1_size/2, fc1_size/8,  1]
dropout_rates = [0.25, 0.25, 0.5, 0]

monitor_mode = 'max'
monitor_metric = 'val_plcc_tf'
outputs = ['qualityScore']

# MODEL DEF
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
import keras

input_feats = Input(shape=input_size, dtype='float32')

# SINGLE-block
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

# pred_aesthetic = apps.fc_layers(x, name       = 'head_aesthetic',
#                                 fc_sizes      = fc_sizes,
#                                 dropout_rates = dropout_rates,
#                                 batch_norm    = bn)

pred_quality = apps.fc_layers(x, name       = 'head_quality',
                              fc_sizes      = fc_sizes,
                              dropout_rates = dropout_rates,
                              batch_norm    = bn)                 

model = Model(inputs=input_feats, outputs=pred_quality) #

gen_params = dict(batch_size    = 128,
                  data_path     = features_file,                  
                  input_shape   = input_size,
                  inputs        = ['sessionId_imageName'],
                  outputs       = outputs, 
                  random_group  = False,
                  fixed_batches = True)

helper = mh.ModelHelper(model, model_name, ids, 
                     max_queue_size = 128,
                     loss           = loss,
                     metrics        = metrics,
                     monitor_metric = monitor_metric, 
                     monitor_mode   = monitor_mode,
                     multiproc      = False, workers = 1,
#                      multiproc      = True, workers = 3,
                     early_stop_patience = 5,
                     logs_root      = root_path + 'logs_all_user_qualityScore_baseline',
                     models_root    = root_path + 'models_all_user_qualityScore_baseline',
                     gen_params     = gen_params)

helper.model_name.update(fc1 = '[%d]' % fc1_size,
                         im  = image_size,
                         bn  = bn,
                         do  = str(dropout_rates).replace(' ',''),
                         mon = '[%s]' % monitor_metric,
                         ds  = '[%s]' % os.path.split(dataset)[1])

print(helper.model_name())

2023-02-05 09:04:21.064882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 09:04:21.808639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 09:04:21.808829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-05 09:04:21.809336: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]


In [4]:
for lr in [1e-4,1e-5,1e-6]:
    helper.load_model()
    helper.train(lr=lr, epochs=20)

Model NOT loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5 does not exist
Training model: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]
Epoch 1/20


2023-02-05 09:05:24.743915: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-02-05 09:05:31.706430: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


170/170 [==============================] - 787s 4s/step - loss: 12.9360 - MAE: 3.2269 - plcc_tf: 0.2275 - val_loss: 9.4967 - val_MAE: 3.0506 - val_plcc_tf: 0.6079
Epoch 2/20
170/170 [==============================] - 693s 4s/step - loss: 10.4674 - MAE: 2.9163 - plcc_tf: 0.2625 - val_loss: 7.3148 - val_MAE: 2.6711 - val_plcc_tf: 0.6793
Epoch 3/20
170/170 [==============================] - 423s 2s/step - loss: 8.6932 - MAE: 2.6306 - plcc_tf: 0.2673 - val_loss: 6.1956 - val_MAE: 2.4498 - val_plcc_tf: 0.7354
Epoch 4/20
170/170 [==============================] - 301s 2s/step - loss: 7.2067 - MAE: 2.3610 - plcc_tf: 0.2781 - val_loss: 4.6191 - val_MAE: 2.1074 - val_plcc_tf: 0.7278
Epoch 5/20
170/170 [==============================] - 318s 2s/step - loss: 5.8305 - MAE: 2.0791 - plcc_tf: 0.2787 - val_loss: 3.6450 - val_MAE: 1.8607 - val_plcc_tf: 0.7294
Epoch 6/20
170/170 [==============================] - 425s 3s/step - loss: 4.7073 - MAE: 1.8131 - plcc_tf: 0.2652 - val_loss: 2.5417 - val_MAE: 

## Test without augmentation

In [6]:
print('After training')

helper.load_model()
test_gen = helper.make_generator(ids[ids.set=='test'])
helper.validate(test_gen, verbose=2)

After training
Model weights loaded: irnv2_mlsp_wide_orig/bn:2 bsz:128 do:[0.25,0.25,0.5,0] ds:[PARA_MTAIQ_All_User_official_da fc1:[2048] i:1[5,5,16928] im:[orig] l:[MSE] mon:[val_plcc_tf] o:1[1]_best_weights.h5
Validating performance
36/36 - 149s - loss: 0.0632 - MAE: 0.1978 - plcc_tf: 0.9168 - 149s/epoch - 4s/step
MAE:     0.1978052407503128
loss:    0.06322509795427322
plcc_tf: 0.9168059825897217


{'loss': 0.06322509795427322,
 'MAE': 0.1978052407503128,
 'plcc_tf': 0.9168059825897217}